# BLU10 - Learning Notebook - Part 2 of 3 - Vectorization

In [1]:
import numpy as np

from statistics import mean
from math import nan, isnan

# 1 Vectorization

Python is a high-level language, valuing convenience more than more control over program execution and, ultimately, performance.

Now, as we've seen, recommender systems deal with large amounts of data, thus emphasizing execution and performance.

NumPy (and Pandas, done right) allows us to have both convenience and performance, using *vectorization*. Curious?

Remember the rating matrix $R$, from the previous notebook:

$$\begin{bmatrix}1 &  & 2\\ 1 & 5 & \\  & 2 & 1\end{bmatrix}$$

Let's start by representing it as a list of lists.

In [2]:
R = [[1, nan, 2], [1, 5, nan], [nan, 2, 1]]
R

[[1, nan, 2], [1, 5, nan], [nan, 2, 1]]

Now, we want to compute the mean rating per user.

In [3]:
def mean_user_ratings(R):
    return [mean([r for r in u if not isnan(r)]) for u in R]


mean_user_ratings(R)

[1.5, 3, 1.5]

In this implementation, Python iterates over the 3 rows, plus 3 elements per row, for a total of 9 cycles.

Vectorization, on the other hand, uses Single Instruction Multiple Data ([SIMD](https://en.wikipedia.org/wiki/SIMD), in short) available in most modern CPUs, to:
* Perform an operation
* On multiple data points
* Simultaneously (i.e., in single a cycle).

Vectorization implements what is known as data parallelism, by applying the same transformation to multiple data in parallel.

Again, Python gives us no control over how a program gets executed, to exploit SIMD directly.

The good news is that NumPy implements vectorization for us.

Features such as array methods and universal functions are used to vectorize operations and remove `for` loops.

# 2 Loading the matrix

Since most times we deal with plain text data, we will use `genfromtxt` to load data from `ratings_matrix.csv`.

In [4]:
R_ = np.genfromtxt('../data/interim/ratings_matrix.csv',  delimiter=',')
R_

array([[ 1., nan,  2.],
       [ 1.,  5., nan],
       [nan,  2.,  1.]])

Just to make sure, let's print of the attributes of the array.

In [5]:
ndims = R_.ndim
nrows = R_.shape[0]
ncols = R_.shape[0] 
dtype = R_.dtype

print("R_ is a {}-dimensional, {} by {} matrix, of {} elements.".format(ndims, nrows, ncols, dtype))

R_ is a 2-dimensional, 3 by 3 matrix, of float64 elements.


# 3 Indexing

## 3.1 User vectors

Sometimes, we want to select vectors of user ratings, such as $R_u = \begin{bmatrix}r_{u, i_1} & ... & r_{u, i_n}\end{bmatrix}$.

Using the square brackets notation and the row index, we do:

```
    ┌───┬───┬───┐   
    │ 1 │   │ 2 │ 
    ┏━━━┳━━━┳━━━┓          ┏━━━┳━━━┳━━━┓
R = ┃ 1 ┃ 5 ┃   ┃ → R[1] = ┃ 1 ┃ 5 ┃   ┃
    ┗━━━┻━━━┻━━━┛          ┗━━━┻━━━┻━━━┛
    │   │ 2 │ 1 │          (view, shape=(3,))
    └───┴───┴───┘
```

In [6]:
R_[1]

array([ 1.,  5., nan])

This returns a *view* of the original array, meaning that modifying it modifies the base array.

We can check whether or not the resulting array is a view using the attribute `ndarray.base`.

In [7]:
R_[1].base is R_

True

However, regular indexing will return a rank-1 array, i.e. 1-dimensional, which in many cases is not desirable.

In [8]:
R_[1].shape

(3,)

If we want to return a *copy* instead of a view of original array, we need to use advanced indexing.

```
    ┌───┬───┬───┐   
    │ 1 │   │ 2 │ 
    ┏━━━┳━━━┳━━━┓            ┏━━━┳━━━┳━━━┓
R = ┃ 1 ┃ 5 ┃   ┃ → R[[1]] = ┃ 1 ┃ 5 ┃   ┃
    ┗━━━┻━━━┻━━━┛            ┗━━━┻━━━┻━━━┛
    │   │ 2 │ 1 │            (copy, shape=(1, 3))
    └───┴───┴───┘
```

In [9]:
R_[[1]]

array([[ 1.,  5., nan]])

In [10]:
R_[[1]].base is R_

False

Do see any other difference?

In [11]:
R_[[1]].shape

(1, 3)

Unless you explicit want to change the array, advanced indexing is recommended.

## 3.2 Item vectors

We use square brackets with comma separated wildcard and column index.

```
    ┌───┬───┏━━━┓             ┏━━━┓
    │ 1 │   ┃ 2 ┃             ┃ 2 ┃
    ├───┼───┣━━━┫             ┣━━━┫
R = │ 1 │ 5 ┃   ┃ → R[:, 2] = ┃   ┃ (view, shape=(3,))
    ├───┼───┣━━━┫             ┣━━━┫
    │   │ 2 ┃ 1 ┃             ┃ 1 ┃
    └───┴───┗━━━┛             ┗━━━┛
```

In [12]:
R_[:, 2]

array([ 2., nan,  1.])

In [13]:
R_[:, 0].base is R_

True

In [14]:
R_[:, 0].shape

(3,)

Again, we should use fancy indexing:

```
    ┌───┬───┏━━━┓               ┏━━━┓
    │ 1 │   ┃ 2 ┃               ┃ 2 ┃
    ├───┼───┣━━━┫               ┣━━━┫
R = │ 1 │ 5 ┃   ┃ → R[:, [2]] = ┃   ┃ (copy, shape=(3, 1))
    ├───┼───┣━━━┫               ┣━━━┫
    │   │ 2 ┃ 1 ┃               ┃ 1 ┃
    └───┴───┗━━━┛               ┗━━━┛
```

In [15]:
R_[:, [2]]

array([[ 2.],
       [nan],
       [ 1.]])

In [16]:
R_[:, [2]].base is R_

False

In [17]:
R_[:, [2]].shape

(3, 1)

## 3.3 Ratings

To select individual elements of our ratings matrix, we combine the notations above, so that:

```
    ┏━━━┓───┬───┐             ┏━━━┓
    ┃ 1 ┃   │ 2 │ → R[0, 0] = ┃ 1 ┃ (scalar)
    ┗━━━┛───┼───┤             ┗━━━┛
R = │ 1 │ 5 │   │  
    ├───┼───┼───┤
    │   │ 2 │ 1 │
    └───┴───┴───┘
```

When selecting individual elements, there are no advantages in using fancy indexing.

In [18]:
R_[0, 0]

1.0

## 3.4 Boolean masks

We use boolean arrays to select specific locations according to a condition.

A boolean mask is an array of boolean values.

```
                  ┌───┬───┬───┐
                  │ 0 │ 1 │ 0 │
                  ├───┼───┼───┤                    
M = np.isnan(R) = │ 0 │ 0 │ 1 │ 
                  ├───┼───┼───┤                             
                  │ 1 │ 0 │ 0 │                   
                  └───┴───┴───┘  
```
That can be used

```
    ┌───┬───┬───┐
    │ 1 │   │ 2 │
    ├───┼───┼───┤          ┌───┬───┬───┐
R = │ 1 │ 5 │   │ → R[M] = │   │   │   │
    ├───┼───┼───┤          └───┴───┴───┘       
    │   │ 2 │ 1 │                   
    └───┴───┴───┘                   
```

In [19]:
M = np.isnan(R_)
M

array([[False,  True, False],
       [False, False,  True],
       [ True, False, False]])

In [20]:
R_[M]

array([nan, nan, nan])

## 3.5 Reshufling vectors

We use these indexing techniques to change the order of the elements of an array.

```
    ┏━━━┓                  ┏━━━┓
    ┃ 0 ┃                  ┃ 1 ┃
    ┣━━━┫                  ┣━━━┫
v = ┃ 1 ┃ → v[[1, 2, 0]] = ┃ 2 ┃
    ┣━━━┫                  ┣━━━┫
    ┃ 2 ┃                  ┃ 0 ┃
    ┗━━━┛                  ┗━━━┛
```

Consider the array `v`. representing a column vector.

In [21]:
v = np.array([[0], [1], [2]])
v

array([[0],
       [1],
       [2]])

We pass an array of indexes to change positions, in a vectorized way.

In [22]:
v[[1, 2, 0]]

array([[1],
       [2],
       [0]])

# 4 Array methods

NumPy arrays have many methods ([docs](https://docs.scipy.org/doc/numpy-1.13.0/reference/arrays.ndarray.html)), which operate on or with the array and typically return a new array.

Such methods can be grouped accorting to their purpose:
* Array conversion
* Shape manipulation
* Item selection and manipulation
* Calculation.

Some common operations we can perform along either axis of a `ndarray` are:
* [argmin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.argmin.html)
* [min](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.min.html)
* [max](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.max.html)
* [round](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.round.html)
* [sum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.sum.html)
* [cumsum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.cumsum.html)
* [mean](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.mean.html)
* [var](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.var.html)
* [std](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.std.html)
* [prod](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.prod.html)
* [cumprod](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.cumprod.html)
* [all](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.all.html)
* [any](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.any.html).

We will illustrate a few methods, particularly relevant to our course.

## 4.1 Reshaping

We can change the shape of the array using `ndarray.reshape`, given the new shape is compatible with the original one.

```
    ┌───┬───┬───┐
    │ 1 │   │ 2 │
    ├───┼───┼───┤                     ┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
R = │ 1 │ 5 │   │ → R.reshape(1, 9) = │ 1 │   │ 2 │ 1 │ 5 │   │   │ 2 │ 1 │
    ├───┼───┼───┤                     └───┴───┴───┴───┴───┴───┴───┴───┴───┘            
    │   │ 2 │ 1 │                   
    └───┴───┴───┘                   
```

We use `reshape` throughout the code to enforce a given shape.

In [23]:
R_.reshape(1, 9)

array([[ 1., nan,  2.,  1.,  5., nan, nan,  2.,  1.]])

## 4.2 Transposing

Other useful operation is transposing a matrix as in linear algebra.

```
    ┌───┬───┬───┐                   ┌───┬───┬───┐
    │ 1 │   │ 2 │                   │ 1 │ 1 │   │
    ├───┼───┼───┤                   ├───┼───┼───┤
R = │ 1 │ 5 │   │ → R.transpose() = │   │ 5 │ 2 │
    ├───┼───┼───┤                   ├───┼───┼───┤                    
    │   │ 2 │ 1 │                   │ 2 │   │ 1 │
    └───┴───┴───┘                   └───┴───┴───┘
```

In [24]:
R_.transpose()

array([[ 1.,  1., nan],
       [nan,  5.,  2.],
       [ 2., nan,  1.]])

## 4.3 Argmax

Returns the indices of the maximum values along an axis (0 for columns, 1 for rows).


```
    ┌───┬───┬───┐
    │ 1 │   │ 2 │
    ├───┼───┼───┤                             ┌───┬───┬───┐
R = │ 1 │ 5 │   │ → np.nanargmax(R, axis=1) = │ 2 │ 1 │ 1 │
    ├───┼───┼───┤                             └───┴───┴───┘            
    │   │ 2 │ 1 │                   
    └───┴───┴───┘                   
```

There is, however, a problem: `argmax` returns `np.NaN` values as maximum values.

In [25]:
R_.argmax(axis=1)

array([1, 2, 0])

Fortunately, NumPy provides a `nanargmax` method to deal with such cases.

In [26]:
np.nanargmax(R_, axis=1)

array([2, 1, 1])

## 4.4 Argsort

Returns the indices that would sort an array along an axis.

Unfortunately, NumPy doesn't provide (to our knowledge) a method for missing values.

Also, `argsort` doesn't provide descending order. So we have to improvise!

```
    ┌───┬───┬───┐                        ┌───┬───┬───┐
    │ 1 │   │ 2 │                        │ 1 │ 0 │ 2 │
    ├───┼───┼───┤                        ├───┼───┼───┤
R = │ 1 │ 5 │   │ → R[np.isnan(R)] = 0 → │ 1 │ 5 │ 0 │
    ├───┼───┼───┤                        ├───┼───┼───┤            
    │   │ 2 │ 1 │                        │ 0 │ 2 │ 1 │
    └───┴───┴───┘                        └───┴───┴───┘ 
    
    ┌───┬───┬───┐                        ┌───┬───┬───┐
    │ 1 │ 0 │ 2 │                        │-1 │ 0 │-2 │
    ├───┼───┼───┤                        ├───┼───┼───┤
R = │ 1 │ 5 │ 0 │ → np.negative(R)     = │-1 │-5 │ 0 │
    ├───┼───┼───┤                        ├───┼───┼───┤            
    │ 0 │ 2 │ 1 │                        │ 0 │-2 │-1 │
    └───┴───┴───┘                        └───┴───┴───┘

    ┌───┬───┬───┐                        ┌───┬───┬───┐
    │-1 │ 0 │-2 │                        │ 2 │ 0 │ 1 │
    ├───┼───┼───┤                        ├───┼───┼───┤
R = │-1 │-5 │ 0 │ → R.argsort(axis=1)  = │ 1 │ 0 │ 2 │
    ├───┼───┼───┤                        ├───┼───┼───┤            
    │ 0 │-2 │-1 │                        │ 1 │ 2 │ 0 │
    └───┴───┴───┘                        └───┴───┴───┘ 
```

In [27]:
R_zeros = R_.copy()
R_zeros[np.isnan(R_)] = 0
R_zeros

array([[1., 0., 2.],
       [1., 5., 0.],
       [0., 2., 1.]])

In [28]:
R_zeros = np.negative(R_zeros)
R_zeros

array([[-1., -0., -2.],
       [-1., -5., -0.],
       [-0., -2., -1.]])

In [29]:
R_zeros.argsort(axis=1)

array([[2, 0, 1],
       [1, 0, 2],
       [1, 2, 0]])

## 4.5 Mean

Finally, we compute the mean rating per user with vectorization.

However, and this is becoming a common theme, 

The `mean` method doesn't deal with missing values, so we use `np.nanmean`.

```
    ┌───┬───┬───┐                                          ┌───┐
    │ 1 │   │ 2 │                                          │1.5│
    ├───┼───┼───┤                                          ├───┤
R = │ 1 │ 5 │   │ → np.nanmean(R, axis=1, keepdims=True) = │ 3 │
    ├───┼───┼───┤                                          ├───┤      
    │   │ 2 │ 1 │                                          │1.5│
    └───┴───┴───┘                                          └───┘
```

In [30]:
R_.mean(axis=1)

array([nan, nan, nan])

In [31]:
np.nanmean(R_, axis=1, keepdims=True)

array([[1.5],
       [3. ],
       [1.5]])

Not only is vectorization performant, but, thanks to NumPy, it's also expressive.

Many other methods behave just like mean, `min`, `max`, `sum`, `var`, `std`, among others.

# 5 Universal functions

Additionally to the array methods discussed above, NumPY provides many [universal functions](https://docs.scipy.org/doc/numpy/reference/ufuncs.html#available-ufuncs).

We group the methods, among others, as:
* Math functions
* Trigonometric functions
* Comparison functions.

Many of these, are implemented in compiled C code, for performance.

Because of vectorization, these functions are applied, in parallel, to multiple data at once.

Universal funtions operate element-by-element and include (some them we already know):
* [add](https://docs.scipy.org/doc/numpy/reference/generated/numpy.add.html)
* [subtract](https://docs.scipy.org/doc/numpy/reference/generated/numpy.subtract.html)
* [multiply](https://docs.scipy.org/doc/numpy/reference/generated/numpy.multiply.html)
* [divide](https://docs.scipy.org/doc/numpy/reference/generated/numpy.divide.html)
* [negative](https://docs.scipy.org/doc/numpy/reference/generated/numpy.negative.html)
* [positive](https://docs.scipy.org/doc/numpy/reference/generated/numpy.positive.html)
* [power](https://docs.scipy.org/doc/numpy/reference/generated/numpy.power.html)
* [exp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html)
* [log](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log.html)
* [sin](https://docs.scipy.org/doc/numpy/reference/generated/numpy.sin.html)
* [cos](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cos.html)
* [greater](https://docs.scipy.org/doc/numpy/reference/generated/numpy.greater.html), [greater_equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.greater_equal.html)
* [less](https://docs.scipy.org/doc/numpy/reference/generated/numpy.less.html), [less_equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.less_equal.html)
* [equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.equal.html)
* [not_equal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.not_equal.html)
* [maximum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.maximum.html)
* [minimum](https://docs.scipy.org/doc/numpy/reference/generated/numpy.minimum.html).

We exemplify some of these functions. Let's start by creating two different matrices.

```
    ┌───┬───┬───┐       ┌───┬───┬───┐
    │ 3 │ 1 │ 2 │       │ 5 │ 5 │ 2 │ 
    ├───┼───┼───┤       ├───┼───┼───┤
A = │ 1 │ 5 │ 4 │   B = │ 3 │ 3 │ 3 │
    ├───┼───┼───┤       ├───┼───┼───┤   
    │ 4 │ 4 │ 1 │       │ 2 │ 5 │ 4 │
    └───┴───┴───┘       └───┴───┴───┘
```

In [32]:
A = np.array([[3, 1, 2], [1, 5, 4], [4, 4, 1]])
B = np.array([[5, 5, 2], [3, 3, 3], [2, 5, 4]])

## 5.1 Add

We use `add` to add arrays element-wise.

```
┌───┬───┬───┐   ┌───┬───┬───┐   ┌───────┬───────┬───────┐   ┌───┬───┬───┐
│ 3 │ 1 │ 2 │   │ 5 │ 5 │ 2 │   │ 3 + 5 │ 1 + 5 │ 2 + 2 │   │ 8 │ 6 │ 4 │ 
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤
│ 1 │ 5 │ 4 │ + │ 3 │ 3 │ 3 │ → │ 1 + 3 │ 5 + 3 │ 4 + 3 │ = │ 4 │ 8 │ 7 │  
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤   
│ 4 │ 4 │ 1 │   │ 2 │ 5 │ 4 │   │ 4 + 2 │ 4 + 5 │ 1 + 4 │   │ 6 │ 9 │ 5 │
└───┴───┴───┘   └───┴───┴───┘   └───────┴───────┴───────┘   └───┴───┴───┘
```

In [33]:
np.add(A, B)

array([[8, 6, 4],
       [4, 8, 7],
       [6, 9, 5]])

Alternatively, you can use math operators, as you would normally do (`add` is called internally).

In [34]:
A + B

array([[8, 6, 4],
       [4, 8, 7],
       [6, 9, 5]])

The downside of using math operators is that you need to make sure `A` and `B` are `ndarrays`.

If you use `add` it will convert any input array-like object into a `ndarray` prior to performing the operation.

You can apply the same reasoning to any other math function, as subtract, multiply or divide.

## 5.2 Log

We use `log` to apply the natural algorithm, element-wise.

```
    ┌───┬───┬───┐               ┌────────┬────────┬────────┐
    │ 3 │ 1 │ 2 │               │ log(3) │ log(1) │ log(2) │
    ├───┼───┼───┤               ├────────┼────────┼────────┤
A = │ 1 │ 5 │ 4 │ → np.log(A) = │ log(1) │ log(5) │ log(4) │
    ├───┼───┼───┤               ├────────┼────────┼────────┤            
    │ 4 │ 4 │ 1 │               │ log(4) │ log(4) │ log(1) │
    └───┴───┴───┘               └────────┴────────┴────────┘
```

In [35]:
np.log(A)

array([[1.09861229, 0.        , 0.69314718],
       [0.        , 1.60943791, 1.38629436],
       [1.38629436, 1.38629436, 0.        ]])

Important functions operate like log, like `exp`, `sin` and `cos`, among others.

## 5.3 Greater

Compares two arrays element-wise.

```
┌───┬───┬───┐   ┌───┬───┬───┐   ┌───────┬───────┬───────┐   ┌───┬───┬───┐
│ 3 │ 1 │ 2 │   │ 5 │ 5 │ 2 │   │ 3 > 5 │ 1 > 5 │ 2 > 2 │   │ 0 │ 0 │ 0 │ 
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤
│ 1 │ 5 │ 4 │ > │ 3 │ 3 │ 3 │ → │ 1 > 3 │ 5 > 3 │ 4 > 3 │ = │ 0 │ 1 │ 1 │  
├───┼───┼───┤   ├───┼───┼───┤   ├───────┼───────┼───────┤   ├───┼───┼───┤   
│ 4 │ 4 │ 1 │   │ 2 │ 5 │ 4 │   │ 4 > 2 │ 4 > 5 │ 1 > 4 │   │ 1 │ 0 │ 0 │
└───┴───┴───┘   └───┴───┴───┘   └───────┴───────┴───────┘   └───┴───┴───┘
```

In [36]:
np.greater(A, B)

array([[False, False, False],
       [False,  True,  True],
       [ True, False, False]])

In [37]:
A > B

array([[False, False, False],
       [False,  True,  True],
       [ True, False, False]])

## 5.4 Maximum

Returns the maximum of two arrays, element-wise.

```
               ┌───────────┬───────────┬───────────┐   ┌───┬───┬───┐
               │ max(3, 5) │ max(1, 5) │ max(2, 2) │   │ 5 │ 5 │ 2 │ 
               ├───────────┼───────────┼───────────┤   ├───┼───┼───┤
np.max(A, B) → │ max(1, 3) │ max(5, 3) │ max(4, 3) │ = │ 3 │ 5 │ 4 │  
               ├───────────┼───────────┼───────────┤   ├───┼───┼───┤   
               │ max(4, 2) │ max(4, 5) │ max(1, 4) │   │ 4 │ 5 │ 4 │
               └───────────┴───────────┴───────────┘   └───┴───┴───┘
```

In [38]:
np.maximum(A, B)

array([[5, 5, 2],
       [3, 5, 4],
       [4, 5, 4]])

# 6 Broadcasting

All this convient functionality operates on a element-by-element basis. 

What about operations on arrays with different sizes?

## 6.1 Columns

We think of boardcasting as streching an array, so that it matches the dimensions of another.

In two dimensions, this means duplicating rows or columns, so that the dimensions of both arrays match.

```
┌───┐   ┌───┬───┬───┐   ┌───┬───┬───┐   ┌───┬───┬───┐
│ 1 │   │ 1 │ 2 │ 3 │   │ 1 │ 1 │ 1 │   │ 1 │ 2 │ 3 │ 
├───┤   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤
│ 2 │ + │ 4 │ 5 │ 6 │ → │ 2 │ 2 │ 2 │ + │ 4 │ 5 │ 6 │   
├───┤   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤   
│ 3 │   │ 7 │ 8 │ 9 │   │ 3 │ 3 │ 3 │   │ 7 │ 8 │ 9 │
└───┘   └───┴───┴───┘   └───┴───┴───┘   └───┴───┴───┘
(3, 1)      (3, 3)          (3, 3)          (3, 3)
```

In this case, we duplicate columns, so that both matrices become the same shape.

If the number of columns of both matrices aren't multiples, an error will be thrown.

In [39]:
A = np.arange(1, 4).reshape(3, 1)
B = np.arange(1, 10).reshape(3, 3)

np.add(A, B)

array([[ 2,  3,  4],
       [ 6,  7,  8],
       [10, 11, 12]])

## 6.2 Rows

We can do the same to the rows, given that the number of rows in each matrix are multiples.

```
                ┌───┬───┬───┐   ┌───┬───┬───┐   ┌───┬───┬───┐
                │ 1 │ 2 │ 3 │   │ 1 │ 2 │ 3 │   │ 1 │ 2 │ 3 │ 
┌───┬───┬───┐   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤
│ 1 │ 2 │ 3 │ + │ 4 │ 5 │ 6 │ → │ 1 │ 2 │ 3 │ + │ 4 │ 5 │ 6 │   
└───┴───┴───┘   ├───┼───┼───┤   ├───┼───┼───┤   ├───┼───┼───┤   
    (1, 3)      │ 7 │ 8 │ 9 │   │ 1 │ 2 │ 3 │   │ 7 │ 8 │ 9 │
                └───┴───┴───┘   └───┴───┴───┘   └───┴───┴───┘
                    (3, 3)          (3, 3)          (3, 3)
```

In [40]:
A = A.reshape(1, 3)

np.add(A, B)

array([[ 2,  4,  6],
       [ 5,  7,  9],
       [ 8, 10, 12]])